# QAAS: Quantum Agent Annealed Swarm

1000 intents, 48 agents, token-based cost model, CQM with dependency chains. Simulated annealing vs greedy.

Cost = estimated_tokens x token_rate per agent. Story points (Fibonacci: 1-2-3-5-8-13) are reporting-only — the CQM stays token-based. Sprint economics track cost/SP and capacity projections.

Code is organized into modules: `config.py`, `agents.py`, `intents.py`, `model.py`, `solve.py`, `report.py`. This notebook is a thin orchestrator.

## Setup

Install D-Wave Ocean SDK:
```bash
pip install dwave-ocean-sdk
```

Configure your API token:
```bash
dwave config create
```

In [1]:
from agents import build_agent_pool
from config import STORY_POINTS
from intents import generate_intents, build_workflow_chains, DISTRIBUTION
from model import build_cqm
from solve import solve_sa, solve_hybrid, parse_assignments, greedy_solve
from report import print_shift_report, print_comparison

## Define the Swarm

48 agents across 4 cloud model tiers and 8 local models.
Cloud agents have 25 task capacity each, local agents are limited but free.

In [2]:
agents, agent_names = build_agent_pool()

num_agents = len(agent_names)
cloud_cap = sum(a['capacity'] for a in agents.values() if not a['is_local'])
local_cap = sum(a['capacity'] for a in agents.values() if a['is_local'])

print(f"Total agents: {num_agents}")
print(f"Cloud: {sum(1 for a in agents.values() if not a['is_local'])} agents, {cloud_cap} capacity")
print(f"Local: {sum(1 for a in agents.values() if a['is_local'])} agents, {local_cap} capacity")
print(f"Total capacity: {cloud_cap + local_cap}")

Total agents: 48
Cloud: 40 agents, 1000 capacity
Local: 8 agents, 19 capacity
Total capacity: 1019


## Define the Intents

1000 tasks across 6 complexity tiers mapped to Fibonacci story points (1-2-3-5-8-13).
3250 total SP. Workflow dependency chains model real feature-dev, bug-fix, and infra flows.

In [3]:
intents = generate_intents()
workflow_chains = build_workflow_chains(intents)

num_intents = len(intents)
num_chained = sum(1 for t in intents if t['depends'])

print(f"Total tasks: {num_intents}")
print(f"Workflow chains: {len(workflow_chains)} ({num_chained} dependent tasks)")
print(f"  feature-dev: {sum(1 for t, _ in workflow_chains if t == 'feature-dev')}")
print(f"  bug-fix:     {sum(1 for t, _ in workflow_chains if t == 'bug-fix')}")
print(f"  infra:       {sum(1 for t, _ in workflow_chains if t == 'infra')}")

total_sp = sum(t['story_points'] for t in intents)
print(f"\nTotal story points: {total_sp}")
print(f"\n{'Tier':<15} {'Count':>5} {'SP':>4} {'Total SP':>9} {'Min Quality':>12}")
print(f"{'─' * 48}")
for c, count, min_q in DISTRIBUTION:
    sp = STORY_POINTS[c]
    tier_count = sum(1 for t in intents if t['complexity'] == c)
    print(f"{c:<15} {tier_count:>5} {sp:>4} {tier_count * sp:>9} {min_q:>12.2f}")

Total tasks: 1000
Workflow chains: 50 (125 dependent tasks)
  feature-dev: 25
  bug-fix:     15
  infra:       10

Total story points: 3250

Tier            Count   SP  Total SP  Min Quality
────────────────────────────────────────────────
trivial           200    1       200         0.40
simple            300    2       600         0.50
moderate          250    3       750         0.70
complex           150    5       750         0.85
very-complex       70    8       560         0.90
epic               30   13       390         0.95


## Build the CQM

CQM separates objective from constraints. Assignment and capacity are hard constraints;
dependency quality ordering is a soft penalty in the objective.

In [4]:
cqm, x = build_cqm(intents, agents, agent_names)

Building CQM: 1000 tasks x 48 agents
Valid assignments: 41600
Filtered out: 6400
Dependency penalty terms: 65065
Constraints: 1048
Build time: 2.68s


## Solve with Simulated Annealing

Convert CQM to BQM with automatic slack variables, then run SA locally.

In [5]:
sampleset = solve_sa(cqm)
assignments = parse_assignments(sampleset, agent_names)

Converting CQM to BQM...
BQM variables: 41817 (includes slack)
BQM quadratic terms: 20031175

Running simulated annealing on 1000-intent problem...
Solve time: 111.0s
Best energy: 176.31


In [6]:
print_shift_report(assignments, intents, agents, workflow_chains)

  FACTORY FLOOR SHIFT REPORT

  Tasks completed:     999/1000
  Tasks dropped:       1
  Money spent:         $79.89
  Quality targets met: 999/999
  Capacity violations: 0
  Dep violations:      0

────────────────────────────────────────────────────────────
  WORKFLOW CHAIN QUALITY PROGRESSION (sample)
────────────────────────────────────────────────────────────
  feature-dev: gemini-8(0.88) -> gemini-0(0.88) -> gpt5.2-0(0.92) -> claude-7(0.95)
  feature-dev: kimi2.5-3(0.85) -> kimi2.5-2(0.85) -> gpt5.2-4(0.92) -> claude-9(0.95)
  feature-dev: kimi2.5-9(0.85) -> gemini-5(0.88) -> gemini-4(0.88) -> claude-0(0.95)
  bug-fix: kimi2.5-8(0.85) -> gemini-3(0.88) -> gemini-2(0.88)
  bug-fix: kimi2.5-3(0.85) -> gemini-0(0.88) -> gemini-7(0.88)
  bug-fix: llama3.2-3b(0.55) -> kimi2.5-3(0.85) -> kimi2.5-4(0.85)
  infra: gemini-6(0.88) -> gemini-5(0.88) -> claude-2(0.95)
  infra: kimi2.5-5(0.85) -> kimi2.5-9(0.85) -> gpt5.2-4(0.92)
  infra: kimi2.5-8(0.85) -> gemini-3(0.88) -> gpt5.2-2(0.92)
  

## Solve with D-Wave (Quantum)

CQM goes directly to the hybrid solver — no manual BQM conversion needed.
Requires a D-Wave Leap API token.

In [ ]:
# Uncomment to run on real quantum hardware

# sampleset = solve_hybrid(cqm)
# feasible = sampleset.filter(lambda s: s.is_feasible)
# if feasible:
#     assignments = parse_assignments(feasible, agent_names)
#     print_shift_report(assignments, intents, agents, workflow_chains)

## Greedy vs Annealing

Greedy picks the cheapest valid agent per task, first-come first-served. No global optimization, no dependency awareness.

In [7]:
greedy_assignments, greedy_cost = greedy_solve(intents, agents)
print_comparison(assignments, greedy_assignments, greedy_cost, intents, agents)

  HEAD TO HEAD: GREEDY vs ANNEALING

  Metric                    Greedy          Annealing      
  ───────────────────────────────────────────────────────
  Tasks shipped             970             999            
  Tasks dropped             30              1              
  Money spent               $86.06          $79.89         
  Story points              2860            3245           
  Cost per SP               $0.0301         $0.0246        
  Cloud tasks               951             980            
  Local tasks (free)        19              19             
  Dep violations            25              0              

  -> Annealing shipped 29 more tasks
  -> Annealing saved $6.17
  -> Annealing delivered 385 more story points
  -> Annealing had 25 fewer dependency violations
